In [1]:
import pandas as pd
import numpy as np

In [2]:
#Read the file we produce in previous web crawler
training_set = pd.read_csv('all_his_join.csv')
test_set = pd.read_csv('all_his_join.csv')
out_put_player = pd.read_csv('all_his_join.csv')

In [3]:
# There are many field which is nah, use zero to filled in
#(it's a little bit controversial here, the people never try 3PT shoot = the people try but never made)
training_set.fillna(0, inplace = True)
test_set.fillna(0, inplace = True)
out_put_player.fillna(0,inplace = True)

In [4]:
# build three part of data :
#1. Training set
#2. Testing set
#3. Player sheet

clean_train_set = training_set.drop(["Lg","Tm","Player",'MP_y','PTS_y','TRB_y','AST_y','STL_y','BLK_y','WS','WS/48','G_y'], axis=1)
clean_test_set = test_set.drop(["Lg","Tm","Player",'MP_y','PTS_y','TRB_y','AST_y','STL_y','BLK_y','WS','WS/48','G_y'], axis=1)
clean_out_put_player = out_put_player.drop(["Lg","Tm",'MP_y','PTS_y','TRB_y','AST_y','STL_y','BLK_y','WS','WS/48','G_y'], axis=1)
clean_train_set = clean_train_set[(clean_train_set['Season'] != '1979-80') & (clean_train_set['Season'] != '2017-18') ]
clean_test_set = clean_test_set[test_set['Season']=='2017-18']

clean_train_set = clean_train_set.drop(['Season'],axis=1)
clean_test_set = clean_test_set.drop(['Season'],axis=1)

In [5]:
# get dummies

clean_train_set = pd.get_dummies(clean_train_set)
clean_test_set = pd.get_dummies(clean_test_set)

In [6]:
# build a decision tree based on sklearn
from sklearn import linear_model
from sklearn import ensemble
from sklearn import tree

In [7]:
# Remove voting results from our inputs
X = clean_train_set.drop(['First', 'Share','Pts Won','Pts Max'], axis=1)


# get the Share (which is mvp point the player get)
y = clean_train_set['Share']

# Train the decision tree
my_decision_tree = tree.DecisionTreeRegressor()
my_decision_tree.fit(X,y)


DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

In [8]:
# Get the importent Feature 
#(PTS_x : total point in the season)
#(W: win game)
#(the other index reference to https://www.basketball-reference.com/)
importance = pd.DataFrame()
importance['Important Features'] = X.columns
importance['Scores'] = my_decision_tree.feature_importances_

importance.sort_values('Scores', ascending=False).head(10)

,Important Features,Scores
20,PTS_x,0.293803
27,W,0.238465
14,TRB_x,0.087104
15,AST_x,0.045392
18,TOV,0.036275
1,G_x,0.032699
3,MP_x,0.024098
8,3P,0.023643
11,FTA,0.023481
28,L,0.022190


In [9]:
# Evaluate the decision tree
my_decision_tree.score(X,y)

0.9999953720660103

In [10]:
# get the test X
X_test = clean_test_set.drop(['First','Share','Pts Won','Pts Max'], axis=1)

#use the build in function predict to predict the test data
test_pred = my_decision_tree.predict(X_test)

test_results = pd.DataFrame(clean_test_set)
test_results['score'] = test_pred

# we delete the player field in previous step, so we join the player to illustrate which player would get the higher mvp score
final_pd = pd.merge(test_results,out_put_player,on=['Age','G_x','MP_x','FG','FGA','2P','2PA','3PA','3P','FT','FTA','ORB','DRB','TOV','PF','FG%','2P%','3P%','eFG%','FT%','TS%','First','Pts Won','Pts Max','Share','W','L'])


final_pd = final_pd[['Player','score']]
print('Decision Tree: 2017-18 Season MVP prediction:')
final_pd.sort_values('score', ascending=False).head(5)


Decision Tree: 2017-18 Season MVP prediction:


,Player,score
193,James Harden,0.922000
172,Paul George,0.020000
507,Russell Westbrook,0.017000
247,LeBron James,0.010000
15,Giannis Antetokounmpo,0.000035


In [11]:
#The format of final data is slightly different to the regular season data, 
#so we need more precess to fetch the data in correct way
X_test_Final = pd.read_csv('final_file_raw.csv')
X_test_Final.fillna(0, inplace = True)
my_final_player = pd.read_csv('final_with_player.csv')
my_final_player.fillna(0, inplace = True)

In [12]:
# Try again with Random Forest
my_rdfroest = ensemble.RandomForestRegressor(n_estimators=25)
my_rdfroest.fit(X,y)
test_pred = my_rdfroest.predict(X_test)

test_results = pd.DataFrame(clean_test_set)
test_results['score'] = test_pred

# we delete the player field in previous step, so we join the player to illustrate which player would get the higher mvp score
final_pd = pd.merge(test_results,out_put_player,on=['Age','G_x','MP_x','FG','FGA','2P','2PA','3PA','3P','FT','FTA','ORB','DRB','TOV','PF','FG%','2P%','3P%','eFG%','FT%','TS%','First','Pts Won','Pts Max','Share','W','L'])

final_pd = final_pd[['Player','score']]
print('Random Forest: 2017-18 Season MVP prediction:')
final_pd.sort_values('score', ascending=False).head(5)

Random Forest: 2017-18 Season MVP prediction:


,Player,score
193,James Harden,0.750080
142,Kevin Durant,0.227041
247,LeBron James,0.205920
121,Anthony Davis,0.157040
111,DeMarcus Cousins,0.128122


In [13]:
#using Gradient Boosting to predict the result
GBR = ensemble.GradientBoostingRegressor()
GBR.fit(X,y)
GBR_test_pred = GBR.predict(X_test)
GBR_test_results = pd.DataFrame(clean_test_set)
GBR_test_results['score'] = GBR_test_pred
final_pd = pd.merge(GBR_test_results,out_put_player,on=['Age','G_x','MP_x','FG','FGA','2P','2PA','3PA','3P','FT','FTA','ORB','DRB','TOV','PF','FG%','2P%','3P%','eFG%','FT%','TS%','First','Pts Won','Pts Max','Share','W','L'])

final_pd = final_pd[['Player','score']]
print('Gradient Boosting: 2017-18 Season MVP prediction:')
final_pd.sort_values('score', ascending=False).head(5)

Gradient Boosting: 2017-18 Season MVP prediction:


,Player,score
193,James Harden,0.500103
247,LeBron James,0.422109
507,Russell Westbrook,0.197394
121,Anthony Davis,0.126623
262,Nikola Jokic,0.098121


In [14]:
#Decision Tree prediction
#use the build in function predict to predict the test data
final_pred = my_decision_tree.predict(X_test_Final)

test_results_fame = pd.DataFrame(my_final_player)
test_results_fame['score'] = final_pred

test_results_fame = test_results_fame[['Unnamed: 0','score']]
print('Decision Tree: 2017-18 Finals MVP prediction:')
test_results_fame.sort_values('score', ascending=False).head(10)

# The result would be a little be weird, because the 2P, 2PA and 2P% did not record in the Finals statics data
#(many fields are 0) 

Decision Tree: 2017-18 Finals MVP prediction:


,Unnamed: 0,score
14,LeBron James,8.790000e-01
3,Draymond Green,2.580000e-01
13,Patrick McCaw,9.651733e-07
25,Jose Calderon,9.651733e-07
24,Ante Zizic,9.651733e-07
23,Kyle Korver,9.651733e-07
22,Jordan Clarkson,9.651733e-07
21,Jeff Green,9.651733e-07
20,Larry Nance,9.651733e-07
19,Tristan Thompson,9.651733e-07


In [15]:
print(X.columns)

Index(['Age', 'G_x', 'GS', 'MP_x', 'FG', 'FGA', '2P', '2PA', '3P', '3PA', 'FT',
       'FTA', 'ORB', 'DRB', 'TRB_x', 'AST_x', 'STL_x', 'BLK_x', 'TOV', 'PF',
       'PTS_x', 'FG%', '2P%', '3P%', 'eFG%', 'FT%', 'TS%', 'W', 'L'],
      dtype='object')


In [16]:
# What if we trim our variable lists...
X_abbreviated = X[['PTS_x', 'TRB_x', 'AST_x', 'TOV', 'W']]
X_test_Final_abbreviated = X_test_Final[['PTS_x', 'TRB_x', 'AST_x', 'TOV', 'W']]
# Train your basic tree
mod = tree.DecisionTreeRegressor()
mod.fit(X_abbreviated,y)
#mod = ensemble.RandomForestRegressor(n_estimators=250)
#mod.fit(X_abbreviated,y)
test_pred = mod.predict(X_test_Final_abbreviated)
test_results = pd.DataFrame(my_final_player)
test_results['score'] = test_pred
test_results = test_results[['Unnamed: 0','score']]
print('Decision Tree: 2017-18 Finals MVP prediction (PTS, TRB, AST, TOV, W):')
test_results.sort_values('score', ascending=False).head()

Decision Tree: 2017-18 Finals MVP prediction (PTS, TRB, AST, TOV, W):


,Unnamed: 0,score
1,Stephen Curry,0.998000
0,Kevin Durant,0.986000
14,LeBron James,0.704000
2,Klay Thompson,0.000004
25,Jose Calderon,0.000002


In [17]:
#use the build in function "predict" to predict the test data
final_pred = my_rdfroest.predict(X_test_Final)

test_results_fame = pd.DataFrame(my_final_player)
test_results_fame['score'] = final_pred
#print(test_pred)
test_results_fame = test_results_fame[['Unnamed: 0','score']]
print('Random Forest Tree: 2017-18 Finals MVP prediction:')
test_results_fame.sort_values('score', ascending=False).head(10)


Random Forest Tree: 2017-18 Finals MVP prediction:


,Unnamed: 0,score
1,Stephen Curry,0.594400
0,Kevin Durant,0.506720
14,LeBron James,0.366601
3,Draymond Green,0.072360
15,Kevin Love,0.053003
2,Klay Thompson,0.009521
16,J.R. Smith,0.000002
25,Jose Calderon,0.000002
24,Ante Zizic,0.000002
23,Kyle Korver,0.000002


In [18]:
# What if we trim our variable lists...
X_abbreviated = X[['PTS_x', 'TRB_x', 'AST_x', 'TOV', 'W']]
X_test_Final_abbreviated = X_test_Final[['PTS_x', 'TRB_x', 'AST_x', 'TOV', 'W']]
# Train your basic tree
my_rdfroest = ensemble.RandomForestRegressor(n_estimators=25)
my_rdfroest.fit(X_abbreviated,y)
#mod = tree.DecisionTreeRegressor()
#mod.fit(X_abbreviated,y)
#mod = ensemble.RandomForestRegressor(n_estimators=250)
#mod.fit(X_abbreviated,y)
test_pred = my_rdfroest.predict(X_test_Final_abbreviated)
test_results = pd.DataFrame(my_final_player)
test_results['score'] = test_pred
test_results = test_results[['Unnamed: 0','score']]
print('Random Forest: 2017-18 Finals MVP prediction (PTS, TRB, AST, TOV, W):')
test_results.sort_values('score', ascending=False).head()

Random Forest: 2017-18 Finals MVP prediction (PTS, TRB, AST, TOV, W):


,Unnamed: 0,score
1,Stephen Curry,0.720560
0,Kevin Durant,0.658440
14,LeBron James,0.484040
2,Klay Thompson,0.038081
15,Kevin Love,0.019803


In [19]:
#using Gradient Boosting to predict the result
GBR = ensemble.GradientBoostingRegressor()
GBR.fit(X,y)
GBR_test_pred = GBR.predict(X_test_Final)
GBR_test_results = pd.DataFrame(my_final_player)
GBR_test_results['score'] = GBR_test_pred
GBR_test_results = GBR_test_results[['Unnamed: 0','score']]
print('Gradient Boosting: 2017-18 Finals MVP prediction:')
GBR_test_results.sort_values('score', ascending=False).head(5)


Gradient Boosting: 2017-18 Finals MVP prediction:


,Unnamed: 0,score
0,Kevin Durant,0.729660
1,Stephen Curry,0.361357
14,LeBron James,0.333794
3,Draymond Green,0.026112
15,Kevin Love,0.013948


In [20]:
# What if we trim our variable lists...
X_abbreviated = X[['PTS_x', 'TRB_x', 'AST_x', 'TOV', 'W']]
X_test_Final_abbreviated = X_test_Final[['PTS_x', 'TRB_x', 'AST_x', 'TOV', 'W']]
# Train your basic tree
GBR = ensemble.GradientBoostingRegressor()
GBR.fit(X_abbreviated,y)
test_pred = my_rdfroest.predict(X_test_Final_abbreviated)
test_results = pd.DataFrame(my_final_player)
test_results['score'] = test_pred
test_results = test_results[['Unnamed: 0','score']]
print('Gradient Boosting: 2017-18 Finals MVP prediction (PTS, TRB, AST, TOV, W):')
test_results.sort_values('score', ascending=False).head()

Gradient Boosting: 2017-18 Finals MVP prediction (PTS, TRB, AST, TOV, W):


,Unnamed: 0,score
1,Stephen Curry,0.720560
0,Kevin Durant,0.658440
14,LeBron James,0.484040
2,Klay Thompson,0.038081
15,Kevin Love,0.019803
